In [ ]:
# ============================================
# STEP 1: Download EMNIST from Kaggle
# ============================================
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

print("📥 Configuration Kaggle API...")
print("⚠️ IMPORTANT: Download your kaggle.json from https://www.kaggle.com/settings/account")
print("=" * 70)

# For Google Colab: Upload kaggle.json
try:
    from google.colab import files

    print("\n📂 Please upload your kaggle.json file:")
    uploaded = files.upload()

    # Configure Kaggle
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    print("✅ Kaggle configured successfully")

except ImportError:
    print("⚠️ Not running in Colab. Please ensure kaggle.json is in ~/.kaggle/")
    print("   Download from: https://www.kaggle.com/settings/account")

# Download EMNIST dataset
print("\n📊 Downloading EMNIST dataset (~560MB, may take 2-5 minutes)...")
print("=" * 70)

!pip install -q kaggle
!kaggle datasets download -d crawford/emnist

print("\n📦 Extracting dataset...")
!unzip -q emnist.zip -d emnist_data

print("✅ Dataset downloaded and extracted to 'emnist_data/' folder")
print("=" * 70)


In [ ]:
# ============================================
# STEP 2: Load EMNIST Data
# ============================================
DATASET_SPLIT = 'digits'  # Options: 'digits', 'letters', 'balanced', 'byclass', 'bymerge', 'mnist'

# Load training data
train_data = pd.read_csv(f'emnist_data/emnist-{DATASET_SPLIT}-train.csv', header=None)
test_data = pd.read_csv(f'emnist_data/emnist-{DATASET_SPLIT}-test.csv', header=None)

# Separate features and labels
y_train = train_data.iloc[:, 0].values
X_train = train_data.iloc[:, 1:].values

y_test = test_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values

# EMNIST images are 28x28, reshape them
img_size = 28
X_train = X_train.reshape(-1, img_size, img_size, 1)
X_test = X_test.reshape(-1, img_size, img_size, 1)

# Rotate -90° and mirror images (EMNIST specific correction)
X_train = np.rot90(X_train, k=3, axes=(1, 2))
X_train = np.flip(X_train, axis=2)

X_test = np.rot90(X_test, k=3, axes=(1, 2))
X_test = np.flip(X_test, axis=2)

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

num_classes = len(np.unique(y_train))

print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")
print(f"Number of classes: {num_classes}")
print(f"Image shape: {X_train.shape[1:]}")
print(f"Dataset split: {DATASET_SPLIT}")

# Load class mapping if balanced dataset
CLASS_MAPPING = None
if DATASET_SPLIT == 'balanced':
    try:
        mapping_file = 'emnist_data/emnist-balanced-mapping.txt'
        class_mapping = {}

        with open(mapping_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 2:
                    class_idx = int(parts[0])
                    ascii_code = int(parts[1])
                    character = chr(ascii_code)
                    class_mapping[class_idx] = character

        CLASS_MAPPING = class_mapping
        print(f"\n✅ Loaded class mapping: {len(class_mapping)} classes")
        print(f"📤 All Characters: {' '.join([class_mapping[i] for i in sorted(class_mapping.keys())])}")

    except FileNotFoundError:
        print(f"⚠️ Mapping file not found")
        CLASS_MAPPING = {i: str(i) for i in range(num_classes)}

# Visualize samples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i].squeeze(), cmap='gray')
    if CLASS_MAPPING:
        label_char = CLASS_MAPPING.get(y_train[i], str(y_train[i]))
        ax.set_title(f"Label: {y_train[i]} ({label_char})")
    else:
        ax.set_title(f"Label: {y_train[i]}")
    ax.axis('off')
plt.suptitle(f'EMNIST {DATASET_SPLIT.upper()} - Sample Images', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# STEP 3: Build TinyML Model
# ============================================
def build_tiny_model(input_shape=(28, 28, 1), num_classes=10):
    """
    Lightweight CNN optimized for TinyML deployment
    """
    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(16, 3, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = layers.MaxPooling2D()(x)

    # Block 2
    x = layers.Conv2D(32, 3, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.MaxPooling2D()(x)

    # Classifier
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

model = build_tiny_model(input_shape=(28, 28, 1), num_classes=num_classes)
model.summary()

print(f"\n📊 Model Statistics:")
print(f"   Total parameters: {model.count_params():,}")
print(f"   Target classes: {num_classes}")


In [ ]:
# ============================================
# STEP 4: Train Model
# ============================================
print(f"\n{'='*70}")
print("🚀 TRAINING MODEL")
print(f"{'='*70}\n")

# Balanced data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.10),
    layers.RandomZoom(0.10),
    layers.RandomTranslation(0.10, 0.10),
    layers.RandomContrast(0.1),
])

def augment_data(images, labels):
    images = data_augmentation(images, training=True)
    return images, labels

BATCH_SIZE = 64

# Create datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(10000).batch(BATCH_SIZE).map(augment_data).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

print(f"Batch size: {BATCH_SIZE}")
print(f"Max epochs: 40")
print(f"Initial learning rate: 1e-3")
print(f"{'='*70}\n")

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=40,
    callbacks=callbacks,
    verbose=1
)

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history.history['accuracy'], label='train')
ax1.plot(history.history['val_accuracy'], label='val')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Model Accuracy')
ax1.legend()
ax1.grid(True)

ax2.plot(history.history['loss'], label='train')
ax2.plot(history.history['val_loss'], label='val')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Model Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# STEP 5: Evaluate Model
# ============================================
print(f"\n{'='*70}")
print("📊 FINAL EVALUATION")
print(f"{'='*70}\n")

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ Test accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"   Test loss: {test_loss:.4f}")

# Calculate per-class accuracy if mapping exists
if CLASS_MAPPING:
    from sklearn.metrics import classification_report, confusion_matrix

    y_pred = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)

    target_names = [CLASS_MAPPING.get(i, str(i)) for i in range(num_classes)]

    print(f"\n📋 Detailed Classification Report:")
    print("-" * 70)
    report = classification_report(y_test, y_pred_classes, target_names=target_names)
    print(report)

    cm = confusion_matrix(y_test, y_pred_classes)
    per_class_acc = cm.diagonal() / cm.sum(axis=1)

    print(f"\n⚠️ Bottom 5 Classes by Accuracy:")
    worst_5 = np.argsort(per_class_acc)[:5]
    for idx in worst_5:
        char = CLASS_MAPPING.get(idx, str(idx))
        acc = per_class_acc[idx]
        print(f"   Class {idx} ({char}): {acc*100:.1f}%")

print(f"\n{'='*70}\n")

In [ ]:
# ============================================
# STEP 6: Convert to TFLite (INT8 Quantized)
# ============================================
print(f"\n{'='*70}")
print("🔧 CONVERTING TO TFLITE (INT8)")
print(f"{'='*70}\n")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_dataset():
    for i in range(100):
        yield [X_train[i:i+1].astype(np.float32)]

converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

# Save TFLite model
model_filename = f'emnist_{DATASET_SPLIT}_int8.tflite'
with open(model_filename, 'wb') as f:
    f.write(tflite_model)

print(f"✅ TFLite model saved: {model_filename}")
print(f"   Model size: {len(tflite_model) / 1024:.2f} KB")
print(f"{'='*70}\n")

In [ ]:
# ============================================
# STEP 7: Test TFLite Model
# ============================================
print(f"\n{'='*70}")
print("🧪 TESTING TFLITE MODEL")
print(f"{'='*70}\n")

interpreter = tf.lite.Interpreter(model_path=model_filename)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("TFLite Model Details:")
print(f"Input shape: {input_details[0]['shape']}")
print(f"Input type: {input_details[0]['dtype']}")
print(f"Output shape: {output_details[0]['shape']}")
print(f"Output type: {output_details[0]['dtype']}")

def run_tflite_inference(interpreter, input_data, input_details, output_details):
    """Run inference on TFLite model with proper quantization"""
    input_info = input_details[0]
    output_info = output_details[0]

    # Quantize input
    if input_info['dtype'] == np.int8:
        scale, zero_point = input_info['quantization']
        input_data = np.round(input_data / scale + zero_point)
        input_data = np.clip(input_data, -128, 127).astype(np.int8)

    interpreter.set_tensor(input_info['index'], input_data)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_info['index'])

    # Dequantize output for visualization
    if output_info['dtype'] == np.int8:
        scale, zero_point = output_info['quantization']
        output_data = (output_data.astype(np.float32) - zero_point) * scale

    return output_data

# Test on samples
num_test_samples = 10
predictions = []

for i in range(num_test_samples):
    input_data = X_test[i:i+1]
    output_data = run_tflite_inference(interpreter, input_data, input_details, output_details)
    predicted_class = np.argmax(output_data)
    predictions.append(predicted_class)

# Visualize predictions
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_test[i].squeeze(), cmap='gray')
    correct = predictions[i] == y_test[i]
    color = 'green' if correct else 'red'

    if CLASS_MAPPING:
        pred_char = CLASS_MAPPING.get(predictions[i], str(predictions[i]))
        true_char = CLASS_MAPPING.get(y_test[i], str(y_test[i]))
        ax.set_title(f"Pred: {pred_char}, True: {true_char}", color=color)
    else:
        ax.set_title(f"Pred: {predictions[i]}, True: {y_test[i]}", color=color)

    ax.axis('off')
plt.suptitle('TFLite Model Predictions', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\n✅ Accuracy on {num_test_samples} samples: {sum([p == y_test[i] for i, p in enumerate(predictions)]) / num_test_samples * 100:.1f}%")


In [ ]:
# ============================================
# STEP 8: Test with Custom Image (Optional)
# ============================================
print(f"\n{'='*70}")
print("🖼️ TESTING WITH CUSTOM IMAGE")
print(f"{'='*70}\n")

try:
    img = cv2.imread("test.png", cv2.IMREAD_GRAYSCALE)

    if img is not None:
        # Preprocess
        img_resized = cv2.resize(img, (28, 28))
        img_normalized = img_resized / 255.0
        input_img = img_normalized.reshape(1, 28, 28, 1).astype(np.float32)

        # Run inference
        output_data = run_tflite_inference(interpreter, input_img, input_details, output_details)
        predicted_class = np.argmax(output_data)

        # Display result
        plt.imshow(img_normalized, cmap='gray')
        if CLASS_MAPPING:
            pred_char = CLASS_MAPPING.get(predicted_class, str(predicted_class))
            plt.title(f"Predicted: {pred_char} (class {predicted_class})")
        else:
            plt.title(f"Predicted: {predicted_class}")
        plt.axis('off')
        plt.show()

        print(f"✅ Predicted class: {predicted_class}")
    else:
        print("⚠️ Could not load 'test.png'. Skipping custom image test.")
        print("   To test with your own image, place a 'test.png' file in the working directory.")

except Exception as e:
    print(f"⚠️ Error loading custom image: {e}")
    print("   Skipping custom image test.")

print(f"\n{'='*70}")
print("✅ PIPELINE COMPLETE")
print(f"{'='*70}\n")